# Bond Cashflows
Simple GUI to show effect of interest rates on net present value of Bond cashflows.  The cashflows consist of an initial payment (not shown) followed by receipt of coupon payments and principle.  NPV gives the current fair value of the bond.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from startup import np, pd, plt, sns

In [3]:
from startup_gui import widgets, interact, interactive

# Bonds
Work in monthly frequency units

In [63]:
def create_bond(duration=12, coupon_rate=0, coupon_period=1, face_value=100):
    if coupon_rate > 0 and coupon_period > 0:
        n_payments = int(duration / coupon_period)
        coupon_payments = np.ones(n_payments) * face_value * coupon_rate * (coupon_period / 12)
        coupon_t = np.arange(1, n_payments+1) * coupon_period
    else:
        coupon_t, coupon_payments = [], []

    principle_t = np.array([duration])
    principle_payment = np.array([face_value])

    cashflows = pd.Series(
        np.concatenate([coupon_payments, principle_payment]), 
        index=np.concatenate([coupon_t, principle_t]), 
        
    )
    return cashflows

In [64]:
# bond = create_bond(coupon_rate=0.1, coupon_period=6, duration=36) # 10% annual interest, semi-annual coupons
# bond, bond.sum()

In [118]:
def plot_cashflows_npv(bond, annual_rate, period):
    period_rate = annual_rate*period/12
    # print(period_rate, (1+period_rate)**(12/period))
    discount_rate = (1 + period_rate)**(bond.index/period)
    discounted_cashflows = bond/discount_rate
    npv = discounted_cashflows.sum()
    gross_value = bond.sum()
    combined = pd.DataFrame({'Cashflows': bond, 'Discount Rate': discount_rate, 'Discounted': discounted_cashflows})
    combined.index.name = 'Month'
    # display(combined)
    ax = combined[['Cashflows', 'Discounted']].plot(kind='bar', title=f'Gross Vaue: {gross_value}  NPV: {npv:0.2f}')


In [119]:
_ = interact(
    lambda coupon_rate, coupon_period, duration, discount_rate: plot_cashflows_npv(
        create_bond(coupon_rate=coupon_rate, coupon_period=coupon_period, duration=duration), discount_rate, coupon_period
    ),
    coupon_rate=widgets.FloatSlider(0.1, min=0., max=0.2, step=0.01),
    coupon_period=widgets.Dropdown(value=6, options=[1, 2, 3, 4, 6, 12]),
    duration=widgets.Dropdown(value=36, options=[12, 24, 36]),
    discount_rate=widgets.FloatSlider(0.05, min=0., max=0.2, step=0.01)
)

interactive(children=(FloatSlider(value=0.1, description='coupon_rate', max=0.2, step=0.01), Dropdown(descript…

In [76]:
create_bond(duration=36, coupon_rate=0.10, coupon_period=3)

3       2.5
6       2.5
9       2.5
12      2.5
15      2.5
18      2.5
21      2.5
24      2.5
27      2.5
30      2.5
33      2.5
36      2.5
36    100.0
dtype: float64

In [ ]:
1.015